Import des données dans la T_enfants
avec traitement des données multiples a separer avant d'etre integrer inidividuellement 

In [1]:
import psycopg2
import os
import csv
import pandas as pd
import re

def chemin_fichier()-> str :
    """
        Donne le nom du chemin complet du fichier actuel (.py) 
    """
    chemin=os.getcwd() #methode pour tourner avec jupyter
    #chemin= path.dirname(path.abspath(__file__)) #methode d'Alice avec <os>
    #chemin = str(Path(__file__).parent) #ma methode avec <path>
    #print(chemin)
    return chemin

# etablir une connection
def connexion_DB(fichierDB):
    try:
        conn = psycopg2.connect(user = "santa",
                                password = "claus",
                                host = "localhost",
                                port = "5432:5432",
                                database = fichierDB
                                )

        cur = conn.cursor()
        # Print PostgreSQL Connection properties
        #print ( "Proprietes : ",conn.get_dsn_parameters(),"\n")

        # Print PostgreSQL version
        cur.execute("SELECT version();")
        record = cur.fetchone()
        #print("Version de PostgreSQL : ", record,"\n")
        cur.close()
    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
    
    return conn

def deconnexion_DB(conn):
#closing database connection.
    if(conn):
        conn.close()
        print("PostgreSQL connection is closed")


In [2]:
conn=connexion_DB("santa_data")
cursor = conn.cursor()

In [3]:
# transfert des données Enfants dans un DataFrame pour traitement des listes 
if(conn):     
    #suppression des enregistrements de la T_pays avant de refaire tourner le programme
    cursor.execute("DELETE FROM T_enfants")
             
    rq_select='''
            SELECT id_l , nom_enfant ,age, id_p
            FROM T_complete
            ORDER BY id_l
            ;
            '''     
    insert_data_query = f""" 
                        INSERT INTO T_enfants
                        (ID_e,
                        ID_l,
                        Nom_enfant,
                        Age,
                        ID_p) 
                        VALUES (default,%s,%s,%s,%s)
                        """
    
    cursor.execute(rq_select)
    res=cursor.fetchall()
       
    count=0
    for row in res :
        l_enfant=re.split(' et ?| ou | & |/|,', row[1])   
        nb_enfant=len(l_enfant)
        nb_age=0
        if row[2] is not None : #age
            l_age=re.split('\s*,\s*|\s*et\s*|\s*an.?|\s*an ?', row[2])
            nb_age=len(l_age)
            
        if nb_enfant > 1 :
            for i in range(0,nb_enfant) :
                l_insert=[row[0]] #id_l
                l_insert.append(l_enfant[i]) #nom_enfant
                if nb_age>2 :
                    l_insert.append(float(l_age[i]))
                    
                elif nb_age == 2 :
                    l_insert.append(float(l_age[0]))
                    
                elif row[2] is None :
                        l_insert.append(row[2]) # age
            
                l_insert.append(row[3]) #id_p
                t_insert=tuple(l_insert)
                print('M',t_insert)
                
                #insertion dans T_enfant
                cursor.execute(insert_data_query,t_insert)
                conn.commit
                count += cursor.rowcount
                            
        elif nb_enfant==1 :
            l_insert=[row[0],row[1]] #id_l   nom_enfant
            if row[2] is None :
                l_insert.append(row[2]) # age
                
            else :
                l_insert.append(float(l_age[0]))
                
            l_insert.append(row[3]) #id_p
            t_insert=tuple(l_insert)
            print('u',t_insert)
            
            #insertion dans T_enfant
            cursor.execute(insert_data_query,t_insert)
            conn.commit()
            count += cursor.rowcount
                
        
            
    print (count, "Record inserted successfully into T_enfants")
    

u (1, 'THEO', 2.0, 12)
u (2, 'Edouard', 5.0, 17)
u (3, 'Manon', 9.0, 12)
u (4, 'Emmanuel', 2.0, 12)
u (5, 'Camille', 14.0, 12)
u (6, 'Sarah-Jane', 7.0, 9)
u (7, 'Mathew', 12.0, 9)
u (8, 'Maxime', 6.0, 12)
u (9, 'Baptiste', 4.0, 12)
u (10, 'Cynthia', None, 12)
u (11, 'Lorine', 5.0, 12)
u (12, 'Noémie', 11.0, 12)
u (13, 'Mélissa', 6.0, 12)
u (14, 'REMI', 9.0, 12)
u (15, 'Emma', 1.0, 12)
u (16, 'Audrey', 12.5, 12)
M (17, 'Clémence', 6.0, 12)
M (17, 'claire', 9.0, 12)
u (18, 'Enzo', 3.0, 12)
u (19, 'Theophile', 4.0, 12)
u (20, 'Julie', 8.0, 12)
u (21, 'Florence', 10.0, None)
u (22, 'Meguy', 8.0, 12)
u (23, 'Eliane', 8.0, 9)
u (24, 'Flora', 25.0, 4)
u (25, 'Yannis', 4.0, 12)
u (26, 'Corentin', 5.0, 12)
u (27, 'Josselin', 3.0, 12)
u (28, 'Laetitia', None, 12)
u (29, 'Eva', 8.0, 12)
u (30, 'Anais', 8.0, 12)
u (31, 'Marie-Christine', 18.0, 9)
u (32, 'David', 33.0, 12)
u (33, 'Mathieu', 15.0, 12)
u (34, 'Sidjy', 7.0, 6)
u (35, 'Coralie', 10.0, 9)
u (36, 'Julie', 23.0, 12)
u (37, 'Nicolas', 10.0

u (324, 'ANGIE', 6.0, 12)
u (325, 'Tom', 5.0, 12)
u (326, 'Theo', 2.0, 12)
u (327, 'Marie', 7.0, 12)
u (328, 'Léo', 1.0, 12)
u (329, 'Alexandra', 8.5, 9)
u (330, 'Tom', 5.0, 12)
u (331, 'Hugo', None, 12)
u (332, 'Helline', 10.0, 12)
u (333, 'Jeanne', 10.0, 12)
u (334, 'Vincent', 5.0, 12)
u (335, 'Deborah', 9.0, 12)
u (336, 'Maxence', 4.0, 9)
u (337, 'Alexia', 3.0, 12)
u (338, 'Olivier', 2.0, 12)
u (339, 'Alexandro', 5.5, 12)
u (340, 'Lea', 3.0, 12)
u (341, 'Morgane', 5.0, 12)
u (342, 'Alexis', 1.5, 12)
M (343, 'Lise', 3.0, 12)
M (343, 'telma', 3.0, 12)
u (344, 'Camille', 11.0, 12)
u (345, 'Isabelle', 13.0, 12)
u (346, 'Garance', 7.0, 12)
u (347, 'Maryline', 9.0, 12)
u (348, 'Camille', 4.0, 12)
u (349, 'Lyne-c', 26.0, 12)
u (350, 'ALEX', 3.0, 12)
u (351, 'Coraline', 4.5, 12)
u (352, 'Jacques', 50.0, 9)
u (353, 'Florent', 28.0, 6)
u (354, 'Vivien', 7.0, 12)
u (355, 'Mathieu', 10.0, 12)
u (356, 'Marylou', 3.0, 12)
u (357, 'Justin', 6.0, 9)
u (358, 'Samuel', 6.0, 12)
u (359, 'Jordane', 6.0

u (650, 'Sean', 6.0, 12)
u (651, 'Mathilde', 1.5, 12)
u (652, 'Manon', 12.0, 12)
u (653, 'Erwann', 4.0, 12)
u (654, 'Maëva', 2.0, 12)
u (655, 'Emma', 2.0, 12)
M (656, 'Marine', 7.0, 12)
M (656, 'loic', 4.0, 12)
u (657, 'Chanael', 10.0, 9)
u (658, 'Kelly', 11.0, 9)
u (659, 'Chrystel', 7.0, 12)
u (660, 'Chloé', 2.6, 19)
u (661, 'Illena', 3.0, 12)
u (662, 'ALEXANDRE', 7.0, 12)
u (663, 'Coralie', 12.0, 12)
u (664, 'XAVIER', 8.0, 12)
u (665, 'Yoann', 2.0, 9)
u (666, 'Florian', 3.0, 12)
u (667, 'Lindsay', 5.0, 12)
u (668, 'Elorri', 3.0, 12)
u (669, 'Marion', 2.5, 12)
u (670, 'Morgane', 5.5, 12)
u (671, 'Laurie', 4.0, 9)
u (672, 'Montserrat', 7.0, 10)
u (673, 'Glenn', 3.0, 12)
u (674, 'Alicia', 8.0, 9)
u (675, 'Grzegorz', 15.0, 27)
u (676, 'SARAH', 4.0, 12)
u (677, 'Maelys', 2.0, 12)
u (678, 'Damien', 5.0, 12)
u (679, 'Etienne', 7.0, 12)
u (680, 'Kylian', 4.0, 12)
u (681, 'Julie', 6.0, None)
u (682, 'Maude', 17.0, 9)
u (683, 'Valérie', 11.0, 9)
u (684, 'Clément', 6.0, 12)
u (685, 'Ismael', 4.

u (988, 'Ophélie', 11.0, 12)
u (989, 'Malie', 0.5, 12)
u (990, 'Kassandra', 12.0, 12)
u (991, 'Olivier', 8.7, 9)
u (992, 'Fabien', 5.0, 12)
u (993, 'Ewann', 2.5, 12)
u (994, 'Noémie', 7.0, 15)
u (995, 'Kenzo', 3.0, 12)
u (996, 'Astrid', 27.0, 12)
u (997, 'Pauline', 4.0, 12)
u (998, 'Laura', 8.0, 12)
u (999, 'Kerryne', 6.0, 12)
u (1000, 'Mathis', 2.0, 12)
u (1001, 'Manon', 5.0, 12)
u (1002, 'Iseult', 5.0, 9)
u (1003, 'Samantha', None, 33)
u (1004, 'Charlène', 11.0, 9)
u (1005, 'JULIE', 5.0, 12)
u (1006, 'Laurine', 8.0, 12)
u (1007, 'Yanïs', 2.0, 12)
u (1008, 'LUCAS', 5.5, 12)
u (1009, 'Florence', 7.0, 9)
u (1010, 'Robbie', 1.0, 12)
u (1011, 'Charles', 2.0, 12)
u (1012, 'AINOHA', 3.0, 12)
u (1013, 'Laurie', 7.0, 12)
u (1014, 'Sevan', 4.0, 12)
u (1015, 'Rachel', 9.0, 9)
u (1016, 'Thomas', 3.0, 12)
u (1017, 'Kâliane', 7.0, 12)
u (1018, 'Soni', 0.5, 12)
u (1019, 'Stéphanie', None, 9)
u (1020, 'Rima', 24.0, 19)
u (1021, 'Caroline', 13.0, 12)
u (1022, 'Sidonie', 6.0, 12)
u (1023, 'Lucie', 10.

u (1399, 'Oceane', 9.0, 12)
u (1400, 'CORENTIN', 8.0, 12)
u (1401, 'Vanessa', 16.0, 18)
u (1402, 'Paul', 4.0, 12)
u (1403, 'Kalvin', 6.0, 12)
u (1404, 'MICHA', 9.0, 9)
u (1405, 'Louis-Alexandre', 2.0, 9)
u (1406, 'Lorelei', 6.0, 12)
u (1407, 'Brice', 4.0, 12)
u (1408, 'Margot', 5.0, 12)
u (1409, 'Theo', 3.0, 12)
u (1410, 'Amandine', 7.0, 12)
u (1411, 'Damien', 5.0, 12)
u (1412, 'Vanessa', 10.0, 9)
u (1413, 'Marie', 4.0, 12)
u (1414, 'Lora', 2.0, 12)
u (1415, 'Andréa', 6.0, 12)
u (1416, 'Gwénaelle', 11.0, 12)
u (1417, 'Eva', 9.0, 12)
u (1418, 'Taina', 9.0, 12)
u (1419, 'Laurie', 14.0, 9)
u (1420, 'Maxou24', 11.0, 9)
u (1421, 'Laurie', 9.0, 12)
u (1422, 'Chalène', 8.0, 12)
u (1423, 'Lily', 5.0, 12)
u (1424, 'Sanah', 8.0, 12)
u (1425, 'Lilou', 9.0, 12)
u (1426, 'Charlott', 5.0, 6)
u (1427, 'EVA', 3.0, 12)
u (1428, 'Clément', 6.0, 12)
u (1429, 'Enzo', 3.0, 12)
u (1430, 'Elena', 3.0, 12)
u (1431, 'Ashley', 7.0, 9)
u (1432, 'Camille', 6.0, 9)
u (1433, 'Maude', None, 9)
u (1434, 'Alison', 12.

In [4]:
rq_select='''
          SELECT 
              ID_e,
              ID_l,
              nom_enfant,
              age,
              TE.ID_p, 
              Pays,ID_l 
          FROM T_enfants TE LEFT JOIN T_pays TP
          ON TE.ID_p=TP.ID_p
          ORDER BY id_l;
          '''

cursor.execute(rq_select)
res=cursor.fetchall()

print('\nID_e'.ljust(8),'ID_l'.ljust(7),'nom_enfant'.ljust(21),'age'.ljust(15),'ID_p'.ljust(8),'Pays'.ljust(20))
trait="-"*80
print(trait)
for row in res:
    print(str(row[0]).ljust(8),end='')
    print(str(row[1]).ljust(8),end='')
    print(str(row[2]).ljust(25)[0:22],end='')
    print(str(row[3]).ljust(15),end='')
    print(str(row[4]).ljust(8),end='')
    print(str(row[5]).ljust(15))


ID_e    ID_l    nom_enfant            age             ID_p     Pays                
--------------------------------------------------------------------------------
1       1       THEO                  2.0            12      France         
2       2       Edouard               5.0            17      Haïti          
3       3       Manon                 9.0            12      France         
4       4       Emmanuel              2.0            12      France         
5       5       Camille               14.0           12      France         
6       6       Sarah-Jane            7.0            9       Canada         
7       7       Mathew                12.0           9       Canada         
8       8       Maxime                6.0            12      France         
9       9       Baptiste              4.0            12      France         
10      10      Cynthia               None           12      France         
11      11      Lorine                5.0            12      Fra

437     429     Raphaelle             5.0            12      France         
438     430     Gabrielle             9.0            9       Canada         
439     431     Marilyne              24.0           12      France         
440     432     Nathalie              15.0           27      Pologne        
441     433     Marina                3.0            12      France         
442     434     Vincent               4.0            12      France         
443     435     René                  58.0           12      France         
444     436     Ritchy                5.0            16      Guyane française
445     437     Quetin                5.0            12      France         
446     438     Kasandra.kenney-lajoie2.0            9       Canada         
447     439     Alexane               5.0            9       Canada         
448     440     Lilian                4.0            12      France         
449     441     Amélie                11.0           9       Canada        

846     822     Thomas                5.0            12      France         
847     823     ADRIEN                6.0            12      France         
848     824     Catherine             5.0            9       Canada         
849     825     Carol-ann             8.0            9       Canada         
850     826     Nadia                 34.0           12      France         
851     827     Clotilde              1.5            12      France         
852     828     Amaya                 10.0           12      France         
853     829     Mari-eve              9.0            9       Canada         
854     830     Sylvie                40.0           12      France         
855     831     Antoine               11.0           12      France         
856     832     Clémence              11.0           9       Canada         
857     833     Carole                4.0            12      France         
858     834     Océane                3.0            12      France         

1244    1215    tomas                 9.0            12      France         
1245    1216    Tiffany               9.0            12      France         
1246    1217    BASTIEN               4.0            12      France         
1247    1218    Nathan                0.5            12      France         
1248    1219    Johan                 9.0            33      Suisse         
1249    1220    Julie                 7.0            12      France         
1250    1221    Felicia               11.0           9       Canada         
1251    1222    Nacha                 5.0            9       Canada         
1252    1223    Hugo                  5.0            12      France         
1253    1224    Jéremy                3.0            12      France         
1254    1225    Amandine              2.0            6       Belgique       
1255    1226    Sidonie               9.0            9       Canada         
1256    1227    Laura                 8.0            12      France         

1628    1596    Anne-Julie            11.0           9       Canada         
1629    1597    Alexia                2.5            12      France         
1630    1598    CHLOE                 8.0            12      France         
1631    1599    Corentin              4.0            12      France         
1632    1600    Elfy                  8.0            12      France         
1633    1601    Valentine             7.0            12      France         
1634    1602    Andrea                6.0            19      Liban          
1635    1603    Dilan                 4.0            12      France         
1636    1604    Eva                   3.0            12      France         
1637    1605    Laurie                4.5            12      France         
1638    1606    Gwendoline            7.0            12      France         
1639    1607    Marie-pier            8.0            9       Canada         
1640    1608    Aurore                10.0           12      France         

In [5]:
cursor.close()
deconnexion_DB(conn)

PostgreSQL connection is closed
